In [1]:
import numpy as np
import pandas as pd
import csv

from sklearn.utils import shuffle
import torch
from torch.distributions.one_hot_categorical import OneHotCategorical 
import pyro
import pyro.distributions as dist

In [15]:
def disease_model(prob_vec):
    
    if len(prob_vec) != 8:
        print ("Number of disease parameter are six")
        return None
        
    #distribuição de paralisia para doença zulombriga
    paralysis = pyro.sample('paralysis', dist.Bernoulli(prob_vec[0]))
    paralysis = 1 if paralysis.item() == 1.0 else 0
    
    tongue = pyro.sample('tongue', dist.Bernoulli(prob_vec[1]))
    tongue = 1 if tongue.item() == 1.0 else 0
    
    finger = pyro.sample('finger', dist.Bernoulli(prob_vec[2]))
    finger = 1 if finger.item() == 1.0 else 0
    
    member = pyro.sample('member', dist.Bernoulli(prob_vec[3]))
    member = 1 if member.item() == 1.0 else 0

    pain = pyro.sample('pain', dist.Bernoulli(prob_vec[4]))
    pain = 1 if pain.item() == 1.0 else 0
    
    anger = pyro.sample('anger', dist.Bernoulli(prob_vec[5]))
    anger = 1 if anger.item() == 1.0 else 0
    
    eyes = pyro.sample('eyes', dist.Bernoulli(prob_vec[6]))
    eyes = 1 if eyes.item() == 1.0 else 0
    
    skin = pyro.sample('skin', dist.Bernoulli(prob_vec[7]))
    skin = 1 if skin.item() == 1.0 else 0
     
    return [paralysis, tongue, finger, member, pain, anger, eyes, skin]

In [21]:
def generate_data(n_cases = 20):
    
    #Model probabilities of the diseases
    zulombriga_prob = [0.0, 0.6, 0.9, 0.7, 0.0, 0.0, 0.0, 0.0]
    infection_prob = [0.5, 0.90, 0.0, 0.5, 0.0, 0.0, 0.2, 0.5]
    viral_prob = [0.1, 0.0, 0.0, 0.5, 0.95, 0.0, 0.0, 0.0]
    fight_prob = [0.0, 0.0, 0.0, 0.7, 0.5, 0.9, 0.0, 0.1]
    t_virus = [0.0, 0.0, 0.5, 0.0, 0.0, 0.9, 0.0, 0.95]
    chickenpox = [0.1, 0.2, 0.0, 0.0, 0.0, 0.0, 0.95, 0.85]

    cases = []
    
    for c in range(n_cases):
        
        disease_vec = [1, 2, 3, 4, 5, 6]
        diseases_prob = [zulombriga_prob, infection_prob, viral_prob, fight_prob, t_virus, chickenpox]
        disease_dist = OneHotCategorical(torch.tensor([0.16, 0.16, 0.16, 0.16, 0.16, 0.16]))
        d_idx = np.argmax(disease_dist.sample())
        diagnosis = disease_vec[d_idx]
        symptoms = disease_model(diseases_prob[d_idx])
        
        line = symptoms + [diagnosis]
        
        cases.append(line)
        
    
    return cases

In [25]:
def save_to_csv(mylist, filename):
    
    header = ["Paralisia", "Língua Amarela", "Dedo Tremendo", "Perda de Membro", "Dor no Peito", "Raiva Severa","Olho vermelho", "Pele azul", "Diagnóstico"]
#     h = ','.join(header)
    
    with open(filename, 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerow(header)
        for item in mylist:
             
             wr.writerow(item)

In [32]:
cases = generate_data(500)
save_to_csv(cases, "new-cases500.csv")

In [33]:
df=pd.read_csv('new-cases500.csv', sep=',')
df = shuffle(df)

In [34]:
df20 = df[:20]
df20

,Paralisia,Língua Amarela,Dedo Tremendo,Perda de Membro,Dor no Peito,Raiva Severa,Olho vermelho,Pele azul,Diagnóstico
111,0,0,0,1,0,1,0,0,4
243,0,0,0,0,1,0,0,0,3
255,0,0,1,0,0,1,0,1,5
367,0,0,0,0,0,1,0,0,4
303,1,1,0,1,0,0,1,1,2
380,0,0,0,1,1,0,0,0,3
402,0,0,0,0,0,0,1,1,6
54,0,0,0,0,0,1,0,1,5
387,0,0,0,0,0,0,1,1,6
320,0,0,0,0,0,1,0,1,5


In [8]:
df20.to_csv("cases20.csv", index=False)

# New Model for ZumbiHealth

In [2]:
def disease_causes():
    # indicates whether the zombie eats ocasianally eats earth 
    earth = pyro.sample('earth', pyro.distributions.Bernoulli(0.7))
    earth = int(earth.item())
    
    # indicates wheter the zombie was bitten by a another one
    bitten = pyro.sample('bite', pyro.distributions.Bernoulli(0.5))
    bitten = int(bitten.item())
    
    return earth, bitten

def diseases(d_causes):
    
    z_worm, t_virus, bact = 0, 0, 0
    
    if d_causes == (1, 0):
        z_worm = 1 
    elif d_causes == (0, 1):
        t_virus = 1
    elif d_causes == (1, 1):
        bact = 1
    else:
        t_virus = pyro.sample('virus', pyro.distributions.Bernoulli(0.4))
        t_virus = int(t_virus.item())
        
    return z_worm, t_virus, bact

def mediators(diseases):
    
    if diseases == (0, 0, 0):
        return None
    
    blood = pyro.sample('blood', pyro.distributions.Normal(150, 10))
    blood = blood.item()
    blood += 50*diseases[0] + 30*diseases[2]
    
    lungs = pyro.sample('lungs', pyro.distributions.Normal(50, 5))
    lungs = lungs.item()
    lungs += 50*diseases[2] - 30*diseases[1]
    
    return int(blood), int(lungs)

def symptons(mediators):
    
    breath = pyro.sample('breath', pyro.distributions.Normal(90, 5))
    breath = breath.item()
    breath += mediators[1]*2
    
    heart = pyro.sample('heart', pyro.distributions.Normal(100, 10))
    heart = heart.item()
    heart += mediators[1] + (mediators[0]*0.5)
    
    yellow_skin = 1 if mediators[0] > 175 else 0
    anger = 1 if mediators[1] > 80 else 0
    
    return int(breath), int(heart), yellow_skin, anger

In [3]:
 def cases_generator(number_cases = 20):
    cases = []
    for _ in range(number_cases):
        dc = disease_causes()
        di = diseases(dc)
        #does not count health zombies
        while di == (0,0,0):
            dc = disease_causes()
            di = diseases(dc)
        
        m = mediators(di)
        s = symptons(m)
        cases.append([dc, di, m , s])
    
    return cases 


In [4]:
def save_csv_portuguese(mycases, filename):
    header = ["Informação Extra (Mordido ou Terra)", "Doenças", "Exames", "Sintomas", "Tratamento"]
#     h = ','.join(header)
    
    with open(filename, 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerow(header)
        for case in cases:
    
            if case[1][0] == 1:
                treatment = "Zumbidazol"
                case.append(treatment)
            elif case[1][1] == 1:
                treatment = "Zanamivir"
                case.append(treatment)
            elif case[1][2] == 1:
                treatment = "Amoxozumbicilina"
                case.append(treatment)
            wr.writerow(case)

In [5]:
cases = cases_generator(number_cases=500)
cases = shuffle(cases)
cases

[[(0, 1), (0, 1, 0), (152, 19), (123, 202, 0, 0)],
 [(1, 1), (0, 0, 1), (186, 104), (291, 293, 1, 1)],
 [(0, 1), (0, 1, 0), (157, 10), (114, 174, 0, 0)],
 [(1, 0), (1, 0, 0), (193, 48), (180, 240, 1, 0)],
 [(1, 1), (0, 0, 1), (171, 103), (288, 280, 0, 1)],
 [(0, 0), (0, 1, 0), (140, 19), (123, 196, 0, 0)],
 [(0, 1), (0, 1, 0), (151, 16), (130, 193, 0, 0)],
 [(1, 0), (1, 0, 0), (162, 41), (163, 239, 0, 0)],
 [(1, 0), (1, 0, 0), (206, 49), (179, 267, 1, 0)],
 [(1, 0), (1, 0, 0), (205, 40), (171, 238, 1, 0)],
 [(1, 0), (1, 0, 0), (193, 46), (171, 230, 1, 0)],
 [(1, 1), (0, 0, 1), (180, 101), (293, 294, 1, 1)],
 [(0, 1), (0, 1, 0), (163, 27), (142, 196, 0, 0)],
 [(1, 0), (1, 0, 0), (202, 53), (194, 264, 1, 0)],
 [(0, 1), (0, 1, 0), (158, 18), (117, 197, 0, 0)],
 [(0, 1), (0, 1, 0), (130, 22), (137, 176, 0, 0)],
 [(1, 0), (1, 0, 0), (190, 52), (196, 240, 1, 0)],
 [(1, 1), (0, 0, 1), (198, 98), (280, 289, 1, 1)],
 [(1, 0), (1, 0, 0), (184, 53), (191, 243, 1, 0)],
 [(1, 1), (0, 0, 1), (195, 9

In [6]:
save_csv_portuguese(cases, "new_model_cases_500.csv")

In [165]:
b

0

# Text Generation